#Chapter.8 문장 생성하기

문장 생성이란 이전 단어들이 주어졌을 때 그다음으로 올 적절한 단어를 선택하는 것은 반복해 문장을 만드는 과제입니다. 이 장에서는 SKT가 공개한 KoGPT2 모델을 가지고 문장을 생성하는 방법을 살펴봅니다. 아울러 NSMC 데이터셋을 가지고 KoGPT2 모델을 파인튜닝하는 내용 역시 다룹니다.

## <8-1 문장 생성 모델 훑어보기>   

이 절에서는 문장 생성 모델의 아키텍처, 입출력 등 전반을 조망해 보겠습니다.    
***문장 생성(sentence generation)***이란 말 그대로 문장을 만들어 내는 과제입니다. 구체적으로는 이전 단어들, 즉 컨텍스트(프롬프트)가 주어졌을 때 다음 단어로 어떤 단어가 오는 게 적절한지 분류하는 것입니다. 문장 생성 과제에서 모델의 입력은 컨텍스트, 출력은 컨텍스트 다음 토큰의 등장 확률이 됩니다.

예를 들어 컨텍스트가 `안녕`일 때 모델의 입출력은 다음과 같습니다.  
입력은 컨텍스트를 토큰화한 결과(토큰 시퀀스)이며 출력은 다음 토큰에 대한 확률 분포입니다. 이 확률 분포의 길이는 어휘 집합의 크기와 같습니다. 다시 말해 모델은 전체 어휘 각각에 대해 컨텍스트 토큰 시퀀스 바로 다음에 올 토큰으로 얼마나 그럴듯한지를 수치로 나타내는 역할을 한다고 볼 수 있습니다.

<입력>   
안녕   

<다음 토큰>  <등장 확률>  
가 : 0.002  
가격 : 0.0001   
…  
아 : 0.0001    
안 : 0.0002    
안녕 : 0.0003   
…    
하 : 0.2   
하다 : 0.15
하세요 : 0.3   
…    
! : 0.16   
? : 0.18    
…   


다음 그림은 앞에서 든 예시에서 컨텍스트가 `안녕`일 때 모델의 출력, 즉 $P(w|안녕)$을 나타냅니다.  
예시와 그림에서 확인할 수 있듯이 모델은 `안녕` 다음 토큰으로 `하세요`가 가장 그럴듯하다고 예측하고 있습니다. 이를 근거를 우리는 `안녕` 다음 토큰으로 `하세요`를 선택할 수 있습니다. 

<center><그림 - 문서 생성(1)></center>

<p align="center"><img src="https://i.imgur.com/neNUDPT.jpg">  


다음 그림은 `안녕`, `하세요`를 컨텍스트로 해서 모델이 출력한 다음 토큰 확률 분포, $P(w|안녕, 하세요)$를 나타낸 그림입니다. 이번에는 모델이 `안녕`, `하세요` 다음 토큰으로 `!`를 가장 그럴듯하다고 예측하고 있습니다. 이에 우리는 `안녕`, `하세요` 다음 토큰으로 `!`를 선택할 수 있게 됩니다.

<center><그림 - 문서 생성(2)></center>

<p align="center"><img src="https://i.imgur.com/HCvQqqJ.jpg">  


문장 생성 방식은 다음과 같습니다.   

>1. 컨텍스트를 모델에 입력해 다음 토큰 확률, $P(w|context)$를 출력한 뒤 다음 토큰을 선택합니다.  
2. 기존 컨텍스트에 1에서 선택한 다음 토큰을 이어붙인 새로운 컨텍스트를 모델에 입력해서 다음 토큰 확률 분포, 즉 $P(w|new   context)$를 추출하고 또다시 그다음 토큰을 선택합니다.
3. 2를 반복해 다음 토큰을 계속 생성해 나갑니다.

앞선 두 그림에서는 모델이 출력한 확률 분포 가운데 확률값이 가장 높은 단어를 다음 토큰으로 선택했는데요, 다음 토큰을 선택하는 방법은 여러 가지입니다. 이 내용은 이후 인퍼런스 실습 때 자세히 살펴보겠습니다. 이렇게 생성한 토큰 시퀀스를 적당하게 후처리해서 사람이 보기에 좋은 형태로 가공하면 최종 문장 생성 결과가 됩니다.   
이 책의 문장 생성 실습에서는 네이버 영화 리뷰 말뭉치인 NSMC 데이터를 활용해 SKT이 공개한 KoGPT2 모델을 파인튜닝하는 실습을 진행합니다.

### 모델 구조
---
이 책에서 사용하는 문장 생성 모델은 ***언어 모델*** 입니다. 3장에서 이미 살펴봤듯이 컨텍스트(이전 단어들)가 주어졌을 때 다음 단어를 맞히는 방식으로 프리트레인을 수행한 모델입니다.

<center><그림 - 언어 모델></center>

<p align="center"><img src="https://i.imgur.com/2Q3BIkc.png"> 


그런데 문장 생성 과제는 문서 분류, 문서 쌍 분류, 개체명 인식, 질의응답 등 저희가 살펴본 기존 과제들과는 특성이 다릅니다.  
 그 차이를 표로 나타내면 다음과 같습니다.

<center><표 - 문장 생성과 기존 과제의 차이점></center>  
  

|항목|문장 생성|기존 과제|
|:---:|:---:|:---:|
|모델 구조|GPT(트랜스포머의 디코더)|BERT(트랜스포머의 인코더)|
|프리트레인 테스크|다음 단어 맞히기|빈칸 맞히기|
|파인 튜닝|다음 단어 맞히기|각 다운스트림 태스크|

문장 생성 과제와 기존 과제의 가장 큰 차이점은 모델 구조입니다.   
전자는 GPT, 후자는 BERT로 수행합니다. 또 다른 차이점은 파인튜닝 방식입니다. 기존 과제는 '빈칸 맞히기'로 프리트레인을 수행하고 NCMC(문서 분류), KLUE-NLI(문서 쌍 분류), KorQuAD(질의응답) 등 각 세부 분야 다운스트림 태스크 관련 데이터로 파인튜닝합니다. 즉, 프리트레인 태스크와 파인튜닝 태스크가 서로 다릅니다. 하지만 문장 생성 과제는 프리트레인과 파인튜닝 태스크가 '다음 단어 맞히기'로 같습니다.

기존 과제는 프리트레인 모델을 그대로 파인튜닝에 사용하기 어렵습니다. 프리트레인 모델 위에 태스크 모듈을 붙여야 파인튜닝이 가능합니다. 그도 그럴 것이 프리트레인 태스크와 파인튜닝 태스크가 서로 다르기 때문입니다. 하지만 문장 생성 과제는 프리트레인과 파인튜닝 태스크가 동일하므로 프리트레인 모델의 구조 변경 없이 그대로 파인튜닝을 수행할 수 있습니다.

예를 들어 파인튜닝 데이터의 입력 단어 시퀀스가 `이 영화 정말 재미`이고 이번에 `있었어`를 맞혀야 하는 상황이라고 가정해 보겠습니다. 이 경우 이번 시점의 정답인 `있었어`에 해당하는 모델 출력 확률은 높이고 나머지 단어의 확률은 낮아지도록 모델 전체를 업데이트합니다.  
이것이 문장 생성 과제의 파인튜닝입니다.

그림 출처 - ratsgo